## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-12-17-23-07 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...
1,2025-07-12-17-13-04 +0000,nyt,Two Palestinians Are Killed in a West Bank Cla...,https://www.nytimes.com/2025/07/12/world/middl...
2,2025-07-12-17-11-45 +0000,nypost,15-year-old illegal migrant arrested for Long ...,https://nypost.com/2025/07/12/us-news/15-year-...
3,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...
4,2025-07-12-16-58-40 +0000,nypost,College student attacked over NYC parking spot...,https://nypost.com/2025/07/12/us-news/college-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2317/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
35,trump,18
28,tariff,5
18,year,5
47,nyc,5
341,canada,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
3,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...,43
42,2025-07-11-22-51-48 +0000,nypost,Dow drops nearly 300 points after Trump teases...,https://nypost.com/2025/07/11/us-news/dow-drop...,40
38,2025-07-11-23-48-26 +0000,nypost,Judge releases bizarre letter from would-be Tr...,https://nypost.com/2025/07/11/us-news/judge-re...,38
19,2025-07-12-14-00-00 +0000,wsj,The White House has an explanation for the nea...,https://www.wsj.com/politics/policy/trump-new-...,38
9,2025-07-12-16-17-04 +0000,nypost,Widow of killed fire chief not satisfied with ...,https://nypost.com/2025/07/12/us-news/corey-co...,37


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
3,43,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...
36,21,2025-07-12-01-53-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...
19,20,2025-07-12-14-00-00 +0000,wsj,The White House has an explanation for the nea...,https://www.wsj.com/politics/policy/trump-new-...
38,20,2025-07-11-23-48-26 +0000,nypost,Judge releases bizarre letter from would-be Tr...,https://nypost.com/2025/07/11/us-news/judge-re...
4,19,2025-07-12-16-58-40 +0000,nypost,College student attacked over NYC parking spot...,https://nypost.com/2025/07/12/us-news/college-...
23,19,2025-07-12-13-26-21 +0000,nypost,Brooklyn DA rules on criminal charges against ...,https://nypost.com/2025/07/12/us-news/king-of-...
47,18,2025-07-11-20-31-41 +0000,nypost,Trump mourns ‘a lot of young angels’ after sur...,https://nypost.com/2025/07/11/us-news/texas-fl...
18,15,2025-07-12-14-24-32 +0000,nypost,Court throws out plea deal for 9/11 mastermind...,https://nypost.com/2025/07/12/us-news/court-th...
9,14,2025-07-12-16-17-04 +0000,nypost,Widow of killed fire chief not satisfied with ...,https://nypost.com/2025/07/12/us-news/corey-co...
20,14,2025-07-12-14-00-00 +0000,nypost,Christians United for Israel offers support wh...,https://nypost.com/2025/07/12/us-news/christia...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
